In [ ]:
# Data loading (features - springs)

import pandas as pd
import json

json_features_file = open("flickr-spring-features.json")

parsed_json_features = json.load(json_features_file)

df_features = pd.json_normalize(parsed_json_features["features"])

df_features = df_features[["latLng", "rank", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank", "resolutions.ALL.maxRank"]]
df_features = df_features.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank", "resolutions.ALL.maxRank": "maxRank"})

df_features = df_features.dropna(subset=["fnRank"])
df_features = df_features.dropna(subset=["sigRank"])
df_features = df_features.dropna(subset=["maxRank"])

return df_features

Index(['rank', 'fnRank', 'sigRank'], dtype='object')


In [ ]:
# Computation Analysis (computing rank)

import numpy as np

df_scatterplot = arg[["rank", "fnRank", "sigRank", "maxRank"]]

df_scatterplot["combinedRanks"] = np.sqrt(
    df_scatterplot["maxRank"] ** 2 +
    df_scatterplot["fnRank"] ** 2 +
    df_scatterplot["sigRank"] ** 2
)

df_scatterplot['linked'] = df_scatterplot.index.to_series().apply(lambda x: [x])

return df_scatterplot

In [ ]:
# Data pool

In [ ]:
# Vega-Lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "title": "RANK (spring)",
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "combinedRanks", "type": "quantitative"},
    "y": {"field": "rank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 



In [ ]:
# Data transformation (connected to df_features - convert latLng into points and create buffer)

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df_points = arg

df_points = df_points.explode("latLng", ignore_index=True)

df_points["geometry"] = df_points["latLng"].apply(lambda x: Point(x[1], x[0]))  # (lon, lat)

gdf_points = gpd.GeoDataFrame(df_points, geometry="geometry", crs="EPSG:4326")

gdf_points = gdf_points.drop(columns=["latLng"])

gdf_points = gdf_points.to_crs(3857)

gdf_points["geometry"] = gdf_points["geometry"].buffer(50)

gdf_points["value"] = 1

gdf_points['linked'] = gdf_points.index.to_series().apply(lambda x: [x])

gdf_points = gdf_points[["geometry", "value", "linked"]]

gdf_points = gdf_points.to_crs(3395)

gdf_points.metadata = {
    'name': 'pulse'
}

return gdf_points

In [ ]:
# Data Pool

In [ ]:
import utk

uc = utk.OSM.load([40.67187576076156, -74.0703927880446, 40.928446768674455, -73.8413807958497], layers=['parks'])

#parks
json_parks = uc.layers['json'][0]
gdf_parks = uc.layers['gdf']['objects'][0]
gdf_parks.metadata = {
 'name': 'parks',
 'style': 'parks'
}

return gdf_parks

In [ ]:
import geopandas as gpd
from shapely.geometry import box

gdf_zip = gpd.read_file("nyc_zip.geojson")

gdf_zip = gdf_zip[["geometry"]]

gdf_zip = gdf_zip.to_crs("EPSG:4326")

min_lat, max_lat = 40.67187576076156, 40.928446768674455
min_lon, max_lon = -74.0703927880446, -73.8413807958497
bbox = box(min_lon, min_lat, max_lon, max_lat)
bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")

gdf_zip = gdf_zip[gdf_zip.within(bbox_gdf.iloc[0].geometry)]

gdf_zip = gdf_zip.to_crs("3395")

gdf_zip.metadata = {
    'name': 'zip'
}

return gdf_zip

In [ ]:
import utk

uc = utk.OSM.load([40.67187576076156, -74.0703927880446, 40.928446768674455, -73.8413807958497], layers=['water'])

json_water = uc.layers['json'][0]
gdf_water = uc.layers['gdf']['objects'][0]
gdf_water.metadata = {
 'name': 'water',
 'style': 'water'
}

return gdf_water

In [ ]:
# Merge (gdf_water, gdf_zip, gdf_parks, data pool)

In [ ]:
# UTK

In [ ]:
# Data loading (features - summer)

import pandas as pd
import json
import numpy as np

json_features_file = open("flickr-summer-features.json")

parsed_json_features = json.load(json_features_file)

df_scatterplot = pd.json_normalize(parsed_json_features["features"])

df_scatterplot = df_scatterplot[["rank", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank", "resolutions.ALL.maxRank"]]
df_scatterplot = df_scatterplot.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank", "resolutions.ALL.maxRank": "maxRank"})

df_scatterplot = df_scatterplot.dropna(subset=["fnRank"])
df_scatterplot = df_scatterplot.dropna(subset=["sigRank"])
df_scatterplot = df_scatterplot.dropna(subset=["maxRank"])

df_scatterplot["combinedRanks"] = np.sqrt(
    df_scatterplot["maxRank"] ** 2 +
    df_scatterplot["fnRank"] ** 2 +
    df_scatterplot["sigRank"] ** 2
)

df_scatterplot['linked'] = df_scatterplot.index.to_series().apply(lambda x: [x])

return df_scatterplot

In [ ]:
# Data pool

In [ ]:
# Vega-lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "title": "RANK (summer)",
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "combinedRanks", "type": "quantitative"},
    "y": {"field": "rank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 

In [ ]:
# Data loading (features - fall)

import pandas as pd
import json
import numpy as np

json_features_file = open("flickr-fall-features.json")

parsed_json_features = json.load(json_features_file)

df_scatterplot = pd.json_normalize(parsed_json_features["features"])

df_scatterplot = df_scatterplot[["rank", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank", "resolutions.ALL.maxRank"]]
df_scatterplot = df_scatterplot.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank", "resolutions.ALL.maxRank": "maxRank"})

df_scatterplot = df_scatterplot.dropna(subset=["fnRank"])
df_scatterplot = df_scatterplot.dropna(subset=["sigRank"])
df_scatterplot = df_scatterplot.dropna(subset=["maxRank"])

df_scatterplot["combinedRanks"] = np.sqrt(
    df_scatterplot["maxRank"] ** 2 +
    df_scatterplot["fnRank"] ** 2 +
    df_scatterplot["sigRank"] ** 2
)

return df_scatterplot

In [ ]:
# Vega-lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "title": "RANK (fall)",
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "combinedRanks", "type": "quantitative"},
    "y": {"field": "rank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 

In [ ]:
# Data loading (features - winter)

import pandas as pd
import json
import numpy as np

json_features_file = open("flickr-winter-features.json")

parsed_json_features = json.load(json_features_file)

df_scatterplot = pd.json_normalize(parsed_json_features["features"])

df_scatterplot = df_scatterplot[["rank", "resolutions.ALL.fnRank", "resolutions.ALL.sigRank", "resolutions.ALL.maxRank"]]
df_scatterplot = df_scatterplot.rename(columns={"resolutions.ALL.fnRank": "fnRank", "resolutions.ALL.sigRank": "sigRank", "resolutions.ALL.maxRank": "maxRank"})

df_scatterplot = df_scatterplot.dropna(subset=["fnRank"])
df_scatterplot = df_scatterplot.dropna(subset=["sigRank"])
df_scatterplot = df_scatterplot.dropna(subset=["maxRank"])

df_scatterplot["combinedRanks"] = np.sqrt(
    df_scatterplot["maxRank"] ** 2 +
    df_scatterplot["fnRank"] ** 2 +
    df_scatterplot["sigRank"] ** 2
)

return df_scatterplot

In [ ]:
# Vega-lite

{ 
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json", 
  "title": "RANK (winter)",
  "params": [ {"name": "clickSelect", "select": "interval"} ], 
  "mark": { "type": "point", "cursor": "pointer" }, 
  "encoding": { 
    "x": {"field": "combinedRanks", "type": "quantitative"},
    "y": {"field": "rank", "type": "quantitative"}, 
    "fillOpacity": { 
        "condition": {"param": "clickSelect", "value": 1}, 
        "value": 0.3 
    }, 
    "color": { 
      "field": "interacted", 
      "type": "nominal", 
      "condition": {
        "test": "datum.interacted === '1'", "value": "red", "else": "blue"} } 
  }, 
  "config": { "scale": { "bandPaddingInner": 0.2 } } 
} 